In [60]:
import pymongo
import pandas as pd
import matplotlib.pyplot as plt 
from datetime import datetime, timedelta
from Press_21_threshold import *
from error_class import Error


In [61]:
def press_21_alert_generator(start,end):
    press = "Press_21"

    sensor_dict = press_21_quantiles(start,end)

    myclient_global = pymongo.MongoClient(host = "128.121.34.13", connect = True )
    press_db = myclient_global[press]
    end = datetime.now()
    start   = end - timedelta(minutes=60)

    min_max = pd.DataFrame()

    for batch in ['BATCH_1','BATCH_2']:
        collection= press_db[batch]
        projection = {}
        projection['_id'] = 0
        projection['Date'] = 1

        for sensor in sensor_dict.keys():
            projection[sensor] = 1

        QUERY = {"Date": {'$gte': start, '$lt':  end}}
        results = collection.find(QUERY,projection)
        df1 = pd.DataFrame(results).set_index('Date')
        describe = df1.describe().T
        describe = df1.quantile([0.005,0.995], axis = 0)
        # describe = describe[['min','max']]
        min_max = pd.concat([min_max,describe],axis=1)

    # print(min_max)

    error_list = []
    for i in range(len(min_max)):
        name = min_max.columns[i]
        lower = min_max.iloc[i,0]
        upper = min_max.iloc[i,1]
        [lower_limit, upper_limit] = sensor_dict[name]

        if lower < lower_limit and upper > upper_limit:
            lower_error = True
            upper_error = True
        elif lower < lower_limit:
            lower_error = True
            upper_error = False
        elif upper > upper_limit:
            lower_error = False
            upper_error = True
        else:
            lower_error = False
            upper_error = False

        if lower_error or upper_error:
            descr = ""
            if lower_error and upper_error:
                descr = "Sensor registered data both below limit and above upper limit in the last 5 minutes"
            elif lower_error:
                descr = "Sensor registered data below lower limit"
            elif upper_error:
                descr = "Sensor registered data above upper limit"
            error = Error(name,descr)
            error_list.append(error)

    return error_list
    
# print(min_max.iloc[1].name)

                            min       max
Col1_Gibbs_Vrms          0.0002    0.0066
Col1_Gibbs_Apeak         0.5000   32.4000
Col1_Gibbs_Arms          0.2000    4.1000
Col1_Gibbs_Crest         3.0000   15.9000
Col1_Gibbs_Temp         85.8200   86.1800
...                         ...       ...
Prs_LubePostFilterPre  236.6240  353.4571
Prs_LubePreFilterPre   289.8644  408.1764
Rm_LubeBlckInPre       100.5652  181.9047
Rm_LeftLwrLnk_Pre       20.7046   26.6202
Rm_RightLwrLnk_Pre      16.2679   23.6624

[101 rows x 2 columns]
[Col1_Gibbs_Vrms: Sensor registered data below lower limit, Col1_Gibbs_Apeak: Sensor registered data below lower limit, Col1_Gibbs_Arms: Sensor registered data below lower limit, Col1_Gibbs_Crest: Sensor registered data both below limit and above upper limit in the last 5 minutes, Col1_Gibbs_Temp: Sensor registered data below lower limit, Col2_Gibbs_Vrms: Sensor registered data below lower limit, Col2_Gibbs_Apeak: Sensor registered data below lower limit, Col2_Gibbs_A